<a href="https://colab.research.google.com/github/2ovisa/AH2179/blob/main/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Assignment 2

*Create a word matrix from a list of string*

In [1]:
#Code is copied from exercise 4
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# Define the text
lines = [
    'The sun sets over the quiet meadow.',
    'Birds chirp softly in the early morning.',
    'A gentle breeze rustles the autumn leaves.',
    'Stars twinkle brightly in the clear night sky.'
]

# Creat the CountVectorizer instance, the stop_words parameter is set to "english"
# to remove common English stop words like "and", "the", and "a"
vectorizer = CountVectorizer(stop_words='english')

# Convert the text to vectorizer
word_matrix = vectorizer.fit_transform(lines)

# The get_feature_names_out method is used to get the feature names
# (i.e., the individual words) in the word matrix
feature_names = vectorizer.get_feature_names_out()

# The line_names list is created to store the names of each line in the original
# list of strings
line_names = [f'Line {(i + 1):d}' for i, _ in enumerate(word_matrix)]

# A Pandas DataFrame is created using the word_matrix.toarray() method to convert
# the sparse matrix to a dense array
df = pd.DataFrame(data=word_matrix.toarray(), index=line_names,columns=feature_names)

# The head() method is called on the resulting DataFrame to display the first few
# rows of the matrix
df.head()

,autumn,birds,breeze,brightly,chirp,clear,early,gentle,leaves,meadow,morning,night,quiet,rustles,sets,sky,softly,stars,sun,twinkle
Line 1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0
Line 2,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0
Line 3,1,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0
Line 4,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,1,0,1


*Downloading data*

In [2]:
#Code is copied from exercise 4
import pandas as pd
import os

# The path of the dataset
url = 'https://raw.githubusercontent.com/zhenliangma/Applied-AI-in-Transportation/master/Exercise_4_Text_classification/Pakistani%20Traffic%20sentiment%20Analysis.csv'

# Load the data use the pandas
df = pd.read_csv(url)

*The assignment*

In [4]:
#Code is somewhat copied from exercise 4, but modified with a for-loop to run all configurations at the same time. This was made with the help of the AI in google-colab
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
%matplotlib inline
from sklearn.metrics import ConfusionMatrixDisplay as cmd
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB
import os
from sklearn.model_selection import GridSearchCV


#-*-*-*-*-*-*chose different vectorization-*-*-*-*-*-*

#To make it more effiecnt the all vectorizers where put in one variable so that them and the models can be put in a for-loop.
vectorizers = {
    "CountVectorizer": CountVectorizer(ngram_range=(1,2), stop_words ='english', min_df=20),
    "HashingVectorizer": HashingVectorizer(ngram_range=(1, 2), n_features=200),
    "TfidfVectorizer": TfidfVectorizer(
        min_df=20,
        norm='l2',
        smooth_idf=True,
        use_idf=True,
        ngram_range=(1, 1),
        stop_words='english'
                       )
}

#-*-*-*-*-*-*chose different vectorization-*-*-*-*-*-*

x = df['Text'] #convert the text to vectorizer
y = df['Sentiment'] #obtain the label
# split into train/test set
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)


#all 6 models where put in the same variable
models = {
    "LR": (LogisticRegression(max_iter=1000, random_state=0),
            {'C':[0.001, 0.01, 0.1, 1, 10, 100]}
    ),
    "KNN": (KNeighborsClassifier(),
            {'n_neighbors' : [3,5,7,9],
             'weights': ['uniform', 'distance'] }
    ),
    "RF": (RandomForestClassifier(random_state=0),
        {'n_estimators': [100, 200, 300],
         'max_depth': [None, 10, 20, 30],
         'min_samples_split': [2, 5, 10],
         'min_samples_leaf': [1, 2, 4]
         }),
    "XGBoost": (XGBClassifier(),
    {'learning_rate': [0.01, 0.1, 0.2],
      'n_estimators': [100, 200, 300],
        'max_depth': [3, 4, 5]}
    ),
    "SVM" : (SVC(probability=True),
            {'kernel': ['linear', 'rbf', 'poly'],'C': [0.1, 1, 10]}
    ),
    "Naïve Bayes" : (BernoulliNB(),
            {'alpha': [0.1, 0.5, 1],'force_alpha': [True,False]}
    )
}

#prints the results of each iteration
results = []

#the for-loop that for every vectorizer runs thorugh all models
for v_name, vectorizer in vectorizers.items():
  print(f"Vectorizer: {v_name}")

  x_train_vectorized = vectorizer.fit_transform(x_train)
  x_test_vectorized = vectorizer.transform(x_test)

  for m_name, (model, param_grid) in models.items():
    print(f"Model: {m_name}")

    #grid search to find the best parameters for each model
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy')
    grid_search.fit(x_train_vectorized, y_train)

    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(x_test_vectorized)

    best_params = grid_search.best_params_
    print(f"Best params: {best_params}")
    best_score = grid_search.best_score_ #cross validation performance
    print(f"CV Score: {best_score}")
    acc = accuracy_score(y_test, y_pred) #test set accuracy
    print(f"Accuracy: {acc}")


    results.append((v_name, m_name, grid_search.best_params_, best_score, acc))



#(1)LR
# model = LogisticRegression(max_iter=1000, random_state=0)
# param_grid = {
#     'C': [0.001, 0.01, 0.1, 1, 10, 100],
# }

#(2)KNN
# model=KNeighborsClassifier()
# param_grid = {
#     'n_neighbors': [3, 5, 7, 9],
#     'weights': ['uniform', 'distance']
# }

#(3)RF
# model = RandomForestClassifier(random_state=0)
# param_grid = {
#     'n_estimators': [100, 200, 300],
#     'max_depth': [None, 10, 20, 30],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4]
# }

#(4)XGBoost
# model =  XGBClassifier()
# param_grid = {
#     'learning_rate': [0.01, 0.1, 0.2],
#     'n_estimators': [100, 200, 300],
#     'max_depth': [3, 4, 5]
# }


#(5)SVM
#model= SVC(probability=True)
#
## this is an example to use the grid search to find the best parameter for SVM model
## param_grid specifies the hyperparameter grid to search over： kernel types
## ('linear', 'rbf', 'poly') and regularization strength C values（0.1, 1, 10）.
#param_grid = {'kernel': ['linear', 'rbf', 'poly'],'C': [0.1, 1, 10]}
#
##`grid_search` performs a grid search with 5-fold cross-validation and evaluates models based on accuracy.
#grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy')
#
##`fit` method fits the model to the training data, systematically trying out all
## parameter combinations.
#grid_search.fit(x_train_vectorized, y_train)
#
##`best_params` and `best_score` store the best hyperparameters and their
## corresponding accuracy score.
#best_params = grid_search.best_params_
#print(best_params)
#best_score = grid_search.best_score_
#
##The `model` is updated with the best estimator found during the grid search,
## which can be used for further analysis.
#model = grid_search.best_estimator_

#(6)Naïve Bayes models
# model=BernoulliNB()
# param_grid = {'alpha': [0.1, 0.5, 1],'force_alpha': [True,False]}

#-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*

#cmd.from_estimator(
#    model,
#    x_test_vectorized,
#    y_test,
#    display_labels=['Positive','Negative'],
#    cmap='Blues',
#    xticks_rotation='vertical'
#    )
#
##calculate accuracy
#print('The accuracy of the model is: '+str(accuracy_score(y_test,model.predict(x_test_vectorized))))

Vectorizer: CountVectorizer
Model: LR
Best params: {'C': 10}
CV Score: 0.9442838832019385
Accuracy: 0.9478672985781991
Model: KNN
Best params: {'n_neighbors': 9, 'weights': 'distance'}
CV Score: 0.9330254771478236
Accuracy: 0.9502369668246445
Model: RF
Best params: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 100}
CV Score: 0.9543641248046635
Accuracy: 0.9715639810426541
Model: XGBoost
Best params: {'learning_rate': 0.2, 'max_depth': 4, 'n_estimators': 200}
CV Score: 0.9484434533738346
Accuracy: 0.9597156398104265
Model: SVM
Best params: {'C': 1, 'kernel': 'rbf'}
CV Score: 0.9448826225133005
Accuracy: 0.9597156398104265
Model: Naïve Bayes
Best params: {'alpha': 0.5, 'force_alpha': True}
CV Score: 0.9419135076290978
Accuracy: 0.9502369668246445
Vectorizer: HashingVectorizer
Model: LR
Best params: {'C': 10}
CV Score: 0.9525924885431847
Accuracy: 0.9502369668246445
Model: KNN
Best params: {'n_neighbors': 9, 'weights': 'distance'}
CV Score: 0.94429266